In [1]:
import sys
sys.path.append('../')

In [14]:
from utils.ucr_helpers import UCR_Data
data = UCR_Data("Mallat")
# data = UCR_Data("StarLightCurves")
# data = UCR_Data("MedicalImages")

In [15]:
import pandas as pd
meta_df = pd.read_csv("../Data/UCR_Summary.csv")
meta_df["learned_w"] = meta_df["DTW (learned_w) "].apply(lambda xi: int(xi.split("(")[-1].split(")")[0]))
meta_df.index = meta_df["Name"].str.lower()

In [16]:
print(f"Number of classes: {data.n_classes}")
print(f"Number of training samples: {data.X_train.shape[0]}")
print(f"Number of test samples: {data.X_test.shape[0]}")
print(f"Length of time series: {data.X.shape[1]}")

Number of classes: 8
Number of training samples: 55
Number of test samples: 2345
Length of time series: 1024


In [17]:
fig = data.plot_fig()
fig.update_layout(
    template="plotly_dark",
    height=300,
    width=600,
    xaxis=dict(title="Time"),
    yaxis=dict(title="Value"),
    margin=dict(l=20, r=20, t=20, b=20)
)

fig.show()

In [72]:
import models.embedding_models as emb
import models.base_model as bm
import importlib
importlib.reload(bm)
importlib.reload(emb)

from models.embedding_models import MatrixFactorization
similarity_matrix=-MatrixFactorization.handle_scaling(temp, scaled='normal')
type(similarity_matrix)

torch.Tensor

In [81]:

similarity_matrix = MatrixFactorization.get_correlation_matrix(data.X, scaled="normal")
# similarity_matrix = MatrixFactorization.get_dtw_matrix(
#     data.X,
#     max_warping_window=meta_df[meta_df.Name == data.name]["learned_w"].iloc[0],
#     return_similarity=True,
#     scaled="normal"
# )

# px.histogram(scaled_correlations[np.triu_indices_from(correlations, k=1)], nbins=30)

- Add in distribution of similarity values here for various transformations

In [82]:
model, losses, learning_rates = MatrixFactorization.train_MF_model(n_time_series=data.X.shape[0], similarity_matrix=similarity_matrix)

100%|██████████| 300/300 [00:20<00:00, 14.68it/s]


In [83]:
MatrixFactorization.plot_embedding_training(losses, learning_rates, verbose=True)

Final pairwise_loss: 108720.34652948612


## Evaluate with Sklearn

In [84]:
from utils.ucr_helpers import evaluate_model_sklearn
from sklearn.neural_network import MLPClassifier

over_sampling = True
embeddings = model.embeddings.weight.detach().numpy()
train_size = data.X_train.shape[0]
embeddings_train = embeddings[:train_size, :]
embeddings_test = embeddings[train_size:, :]
y_train = data.y[:train_size]
y_test = data.y[train_size:]

evaluate_model_sklearn(
            embeddings_train,
            embeddings_test,
            y_train,
            y_test,
            classifier=MLPClassifier(max_iter=2000),
            over_sampling=over_sampling,
            verbose=True,
        )

              precision    recall  f1-score   support

           1       0.97      0.80      0.88       294
           2       1.00      1.00      1.00       292
           3       0.92      1.00      0.96       294
           4       0.81      0.97      0.88       289
           5       1.00      0.94      0.97       298
           6       1.00      0.97      0.98       294
           7       1.00      1.00      1.00       291
           8       1.00      0.97      0.98       293

    accuracy                           0.96      2345
   macro avg       0.96      0.96      0.96      2345
weighted avg       0.96      0.96      0.96      2345



0.9560767590618336

## Pytorch Classifier

In [11]:
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
import numpy as np

X = model.embeddings.weight.detach()
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data.y)
y = torch.tensor(y, dtype=torch.long)
n_train = data.X_train.shape[0]

# Split the data into training and test sets
X_train = X[:n_train]
y_train = y[:n_train]
X_test = X[n_train:]
y_test = y[n_train:]

if True:
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    ros = RandomOverSampler()
    X_train, y_train = ros.fit_resample(X_train,y_train)
    X_train = torch.tensor(X_train)
    y_train = torch.tensor(y_train)

# Convert to TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
# You can define your own batch_size and whether to shuffle the data
batch_size = 12  # Example batch size
shuffle = True   # Shuffle the training data

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class EntityClassifier(nn.Module):
    def __init__(self, embedding_size, num_classes):
        super(EntityClassifier, self).__init__()
        self.fc1 = nn.Linear(embedding_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.batchnorm = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Function to evaluate the model
def evaluate_model_pytorch(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    all_predictions = []
    all_labels = []

    with torch.no_grad():  # Disable gradient computation
        for embeddings, labels in test_loader:
            outputs = model(embeddings)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_predictions, all_labels

LEARNING_RATE = 0.02
EPOCHS = 50
clf = EntityClassifier(embedding_size=model.embedding_dim, num_classes=data.n_classes)
optimizer = torch.optim.Adam(clf.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.8, patience=10, threshold=0.0001
)
criterion = torch.nn.CrossEntropyLoss()
training = []
for epoch in tqdm(range(EPOCHS)):
    clf.train()
    epoch_loss = 0
    for i, (embeddings, labels) in enumerate(train_loader):
        outputs = clf(embeddings)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss+=loss.item()
    scheduler.step(epoch_loss)


    y_preds, y_true = evaluate_model_pytorch(clf, train_loader)
    train_accuracy = accuracy_score(y_preds, y_true)
    y_preds, y_true = evaluate_model_pytorch(clf, test_loader)
    test_accuracy = accuracy_score(y_preds, y_true)

    training.append((epoch_loss, optimizer.param_groups[0]["lr"], train_accuracy, test_accuracy))





  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [00:01<00:00, 172.01it/s]


In [12]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np



# Evaluate the model
predictions, true_labels = evaluate_model_pytorch(clf, test_loader)

# Print the classification report
print(classification_report(true_labels, predictions, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

           1       1.00      0.83      0.91         6
           2       0.67      0.67      0.67         6
           3       0.80      0.67      0.73         6
           4       0.60      1.00      0.75         6
           5       0.75      0.50      0.60         6

    accuracy                           0.73        30
   macro avg       0.76      0.73      0.73        30
weighted avg       0.76      0.73      0.73        30



In [13]:
total_losses, learning_rates, train_accuracies, test_accuracies = zip(*training)
np.mean(test_accuracies[-10:])

0.7133333333333334

In [14]:
import plotly.graph_objects as go

# Unpack the losses
total_losses, learning_rates, train_accuracies, test_accuracies = zip(*training)

# Create a figure
fig = go.Figure()

# Add traces for pairwise and regularization losses
fig.add_trace(go.Scatter(x=list(range(len(total_losses))), y=pairwise_losses, mode='lines', name='Total Loss'))

# Create a secondary y-axis for the total loss
fig.update_layout(
    yaxis=dict(title='Pairwise and Regularization Loss'),
    yaxis2=dict(title='Learning Rate', overlaying='y', side='right')
)

# Add the total loss trace
fig.add_trace(go.Scatter(x=list(range(len(learning_rates))), y=learning_rates, mode='lines', name='Learning rate', yaxis='y2'))
fig.add_trace(go.Scatter(x=list(range(len(train_accuracies))), y=train_accuracies, mode='lines', name='Train Acc', yaxis='y2'))
fig.add_trace(go.Scatter(x=list(range(len(test_accuracies))), y=test_accuracies, mode='lines', name='Test Acc', yaxis='y2'))

# Update layout
fig.update_layout(title='Losses During Training', xaxis_title='Epoch', yaxis_title='Loss')
fig.update_layout(template='plotly_dark')

# Show the figure
fig.show()

## Visualization

In [15]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def plot_with_dimensionality_reduction(model, data, method='pca'):
    # generate some sample data
    embeddings = model.embeddings.weight.detach().numpy()

    # check the method chosen by the user and apply accordingly
    if method.lower() == 'pca':
        # apply PCA to reduce the dimensionality of the data to 2D
        reduction = PCA(n_components=2)
    elif method.lower() == 'tsne':
        # apply t-SNE to reduce the dimensionality of the data to 2D
        reduction = TSNE(n_components=2)
    else:
        raise ValueError("Invalid method. Choose 'pca' or 'tsne'.")

    reduced_data = reduction.fit_transform(embeddings)

    # generate some sample class labels
    labels = data.y

    # convert the reduced data, class labels, and entity names to a pandas DataFrame for plotting
    df = pd.DataFrame({"x": reduced_data[:, 0], "y": reduced_data[:, 1], "label": labels})
    df = df.sort_values(by="label")

    # plot the data using plotly, colored by the class labels
    fig = px.scatter(df, x="x", y="y", color="label", hover_name="label")
    # Update layout for smaller margins and academic style
    fig.update_layout(
        template="simple_white",  # Simple and clean layout
        margin=dict(l=20, r=20, b=20, t=20),  # Smaller margins
        font=dict(
            family="Arial", size=12, color="black"
        ),  # Academic-style font and color
        xaxis=dict(title=f"{method.upper()} Component 1", title_font=dict(size=14)),
        yaxis=dict(title=f"{method.upper()} Component 2", title_font=dict(size=14)),
        height=300,
        width=500,
        legend=dict(
            title="Class",
            # orientation="h",
            # yanchor="bottom",
            # y=1.02,
            # xanchor="right",
            # x=1,
        ),
    )
    fig.update_traces(marker={"size": 4})
    # fig.update_layout(template='plotly_dark')
    fig.show()


plot_with_dimensionality_reduction(model, data, method='tsne')

/opt/anaconda3/envs/phd/lib/python3.11/site-packages/threadpoolctl.py:1010: RuntimeWarning:


Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md




Write function to plot based on index and show that misclassified points are identical to nearest neighbours? Also there appears to be little subclusters within the group 5 and 3 clusters

### Grid search to find best params

In [30]:
import itertools
import torch
from tqdm import tqdm
from models.embedding_models import MatrixFactorization
from typing import List, Tuple


# Define the hyperparameter grid
hyperparameter_grid = {
    'embedding_dim': [30,50,100],
    'learning_rate': [0.05],
    'epochs': [300],
    # 'regularization_loss_weight': [0.05, 0.1, 0.15],
    # 'pairwise_loss_weight': [0.001, 0.01, 0.1]
}



# Function to train and evaluate the model
def train_and_evaluate(hyperparams, data):
    # Unpack hyperparameters
    embedding_dim, learning_rate, epochs= hyperparams

    # Initialize and train Matrix Factorization model
    model = MatrixFactorization(n_time_series=data.X.shape[0], embedding_dim=embedding_dim, normalize=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.8, patience=10, threshold=0.1
    )

    for epoch in range(epochs):
        optimizer.zero_grad()
        total_loss, pairwise_loss, regularization_loss = calculate_losses(model, correlations)
        total_loss.backward()
        optimizer.step()
        scheduler.step(pairwise_loss)

    # Prepare data for evaluation
    embeddings = model.embeddings.weight.detach().numpy()
    train_size = data.X_train.shape[0]
    embeddings_train = embeddings[:train_size, :]
    embeddings_test = embeddings[train_size:, :]
    y_train = data.y[:train_size]
    y_test = data.y[train_size:]

    # Evaluate model
    accuracy = evaluate_model(embeddings_train, embeddings_test, y_train, y_test, over_sampling=True)
    return accuracy

total_combinations = 1
for values in hyperparameter_grid.values():
    total_combinations *= len(values)
# Grid search
results = []
for params in tqdm(itertools.product(*hyperparameter_grid.values()), total=total_combinations):
    performance = train_and_evaluate(params, data)
    print("="*20)
    print(params)
    print(performance)
    results.append((params, performance))

# Find the best performing hyperparameters
best_params = max(results, key=lambda x: x[1])[0]



 33%|███▎      | 1/3 [00:00<00:01,  1.71it/s]

(30, 0.05, 300)
0.7


 67%|██████▋   | 2/3 [00:01<00:00,  1.73it/s]

(50, 0.05, 300)
0.8


100%|██████████| 3/3 [00:02<00:00,  1.47it/s]

(100, 0.05, 300)
0.8333333333333334
